In [1]:
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image
import os
import shutil
from keras import layers
from keras import models
from keras import optimizers
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras import regularizers
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np
from keras.callbacks import Callback
from scipy.stats import sem, t
from scipy import mean
import seaborn as sns
from PIL import Image

Using TensorFlow backend.


In [9]:
inria_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\real_data'
brainwash_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\brainwash\\work'

In [10]:
inria_brain_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\inria_brainwash'

In [11]:
ib_pos_dir = os.path.join(inria_brain_dir, 'pos')
os.mkdir(ib_pos_dir)

In [12]:
ib_neg_dir = os.path.join(inria_brain_dir, 'neg')
os.mkdir(ib_neg_dir)

In [13]:
inria_pos_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\real_data\\pos'
inria_neg_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\real_data\\neg'

In [14]:
brainwash_pos_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\brainwash\\work\\pos'
brainwash_neg_dir = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\brainwash\\work\\neg'

In [15]:
i = 0
for image in os.listdir(brainwash_pos_dir):
    full_path = brainwash_pos_dir + '\\' + image
    img = Image.open(full_path)
    w, h = img.size
    croped_img = img.crop((0, 90, w, h-90))
    croped_img.save(ib_pos_dir + '\\' + str(i) + 'image.png', 'png', quality=80, optimize=True, progressive=True)
    i += 1

In [16]:
i = 0
for image in os.listdir(brainwash_neg_dir):
    full_path = brainwash_neg_dir + '\\' + image
    img = Image.open(full_path)
    w, h = img.size
    croped_img = img.crop((0, 90, w, h-90))
    croped_img.save(ib_neg_dir + '\\' + str(i) + 'image.png', 'png', quality=80, optimize=True, progressive=True)
    i += 1

In [17]:
inria_pos_files = os.listdir(inria_pos_dir)
i = 0
for file_name in inria_pos_files:
    full_file_name = os.path.join(inria_pos_dir, file_name)
    if i<411:
        shutil.copy(full_file_name, ib_pos_dir)
    if i>=411:
        break
    i += 1

In [18]:
new_inria_path = 'C:\\Users\\Anton.Lahunou\\detect_people_task\\INRIAPerson\\Train\\neg'

In [19]:
inria_neg_files = os.listdir(new_inria_path)
i = 0
for file_name in inria_neg_files:
    full_file_name = os.path.join(new_inria_path, file_name)
    if i<1204:
        shutil.copy(full_file_name, ib_neg_dir)
    if i>=1204:
        break
    i += 1

In [20]:
print('Total pos images:', len(os.listdir(ib_pos_dir)))
print('Total neg images:', len(os.listdir(ib_neg_dir)))

Total pos images: 1500
Total neg images: 1500


In [21]:
datagen = ImageDataGenerator(rescale=1./255)

In [22]:
generator = datagen.flow_from_directory(inria_brain_dir, target_size=(150, 150), batch_size=3000, shuffle=True, class_mode='binary')

Found 3000 images belonging to 2 classes.


In [23]:
for data_batch, labels_batch in generator:
    images = data_batch
    labels = labels_batch
    break

In [24]:
k = 6
samples = len(images) // k

In [25]:
def build_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    return model

In [ ]:
train_data = images[samples:]
train_targets = labels[samples:]
test_data = images[:samples]
test_targets = labels[:samples]
model = build_cnn()
history = model.fit(train_data, train_targets, epochs=50, batch_size=64, validation_split=0.2)
results = model.evaluate(test_data, test_targets)

Train on 2000 samples, validate on 500 samples
Epoch 1/50
 832/2000 [===========>..................] - ETA: 2:11 - loss: 11.7364 - acc: 0.5901

In [ ]:
plt.plot(history.history['acc'], 'bo', label='Training acc')
plt.plot(history.history['val_acc'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(history.history['loss'], 'bo', label='Training loss')
plt.plot(history.history['val_loss'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
results